In [1]:
import pandas as pd
import numpy as np
from skimpy import clean_columns
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime
import scipy.stats as stats
from datetime import timedelta

In [38]:
scm_2023 = pd.read_csv('../2023/Data/2023_scm_counts.csv')
scm_2023 = clean_columns(scm_2023)
scm_2023['date'] = pd.to_datetime(scm_2023["date"], format="%m/%d/%y")
scm_2023[['n_scm_i_m', 'n_scm_o_m', 'n_scm_i_f', 'n_scm_o_f']] = scm_2023[['n_scm_i_m', 'n_scm_o_m', 'n_scm_i_f', 'n_scm_o_f']].apply(pd.to_numeric, errors='coerce')
scm_2023['scm_male'] = scm_2023['n_scm_i_m'] + scm_2023['n_scm_o_m']
scm_2023['scm_female'] = scm_2023['n_scm_i_f'] + scm_2023['n_scm_o_f']
scm_2023['total_scm'] = scm_2023['n_scm_i_m'] + scm_2023['n_scm_o_m'] + scm_2023['n_scm_i_f'] + scm_2023['n_scm_o_f']
scm_2023['season_total'] = scm_2023.groupby('site')['total_scm'].transform('sum')
scm_2023['male_total'] = scm_2023.groupby('site')['scm_male'].transform('sum')
scm_2023['female_total'] = scm_2023.groupby('site')['scm_female'].transform('sum')
scm_2023['season_prop'] = scm_2023['total_scm'] / scm_2023['season_total']
scm_2023['male_prop'] = scm_2023['scm_male'] / scm_2023['male_total']
scm_2023['female_prop'] = scm_2023['scm_female'] / scm_2023['female_total']
scm_2023

,card_id,data_collector,site,date,n_scm_i_m,n_scm_i_f,n_scm_o_m,n_scm_o_f,n_d_florilega_i,n_d_florilega_o,...,notes,scm_male,scm_female,total_scm,season_total,male_total,female_total,season_prop,male_prop,female_prop
0,2001,Anna DiPaola,DIP_CUR,2023-03-13,0.0,0.0,0.0,0.0,0,0,...,,0.0,0.0,0.0,1309.0,655.0,654.0,0.000000,0.000000,0.000000
1,2002,Anna DiPaola,DIP_CUR,2023-03-13,0.0,0.0,0.0,0.0,0,0,...,,0.0,0.0,0.0,1309.0,655.0,654.0,0.000000,0.000000,0.000000
2,2003,Anna DiPaola,POV_DUN,2023-03-13,0.0,0.0,0.0,0.0,0,0,...,,0.0,0.0,0.0,275.0,185.0,90.0,0.000000,0.000000,0.000000
3,2004,Anna DiPaola,POV_DUN,2023-03-13,0.0,0.0,0.0,0.0,0,0,...,,0.0,0.0,0.0,275.0,185.0,90.0,0.000000,0.000000,0.000000
4,2005,Anna DiPaola,DIP_FLE,2023-03-13,0.0,0.0,0.0,0.0,0,0,...,,0.0,0.0,0.0,1556.0,780.0,776.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2375,7408,Erik Smith,SMI_MCC,2023-07-22,1.0,1.0,2.0,7.0,0,0,...,NaN,3.0,8.0,11.0,264.0,138.0,126.0,0.041667,0.021739,0.063492
2376,7409,Erik Smith,SMI_JOH,2024-09-25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,497.0,258.0,239.0,NaN,NaN,NaN
2377,7410,Erik Smith,SMI_JOH,2024-09-25,0.0,0.0,0.0,0.0,0,0,...,NaN,0.0,0.0,0.0,497.0,258.0,239.0,0.000000,0.000000,0.000000
2378,7411,Erik Smith,SMI_HOF,2024-09-25,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,444.0,244.0,200.0,NaN,NaN,NaN


In [36]:
scm_2024 = pd.read_csv('../2024/Data/2024_scm_counts.csv')
scm_2024 = clean_columns(scm_2024)
scm_2024['date_collected'] = pd.to_datetime(scm_2024["date_collected"], format='mixed')
scm_2024['scm_male'] = scm_2024['n_scm_m_1'] + scm_2024['n_scm_m_2']
scm_2024['scm_female'] = scm_2024['n_scm_f_1'] + scm_2024['n_scm_f_2']
scm_2024['total_scm'] = scm_2024['n_scm_m_1'] + scm_2024['n_scm_m_2'] + scm_2024['n_scm_f_1'] + scm_2024['n_scm_f_2']
scm_2024['season_total'] = scm_2024.groupby('field_id')['total_scm'].transform('sum')
scm_2024['male_total'] = scm_2024.groupby('field_id')['scm_male'].transform('sum')
scm_2024['female_total'] = scm_2024.groupby('field_id')['scm_female'].transform('sum')
scm_2024['season_prop'] = scm_2024['total_scm'] / scm_2024['season_total']
scm_2024['male_prop'] = scm_2024['scm_male'] / scm_2024['male_total']
scm_2024['female_prop'] = scm_2024['scm_female'] / scm_2024['female_total']
scm_2024

,card_id,data_collector,field_id,date_placed,date_collected,n_scm_m_1,n_scm_f_1,n_scm_m_2,n_scm_f_2,n_d_florilega_1,n_d_florilega_2,initials,scm_male,scm_female,total_scm,season_total,male_total,female_total
0,24_1079,Shea Crowther,CUR_FIE_IN,3/5/24,2024-03-12,0.0,0.0,0.0,0.0,0.0,0.0,SN,0.0,0.0,0.0,131.0,78.0,53.0
1,24_1095,Shea Crowther,CUR_FIE_IN,3/12/24,2024-03-19,1.0,0.0,0.0,0.0,0.0,0.0,SN,1.0,0.0,1.0,131.0,78.0,53.0
2,24_1111,Shea Crowther,CUR_FIE_IN,3/19/24,2024-03-26,0.0,0.0,0.0,0.0,0.0,0.0,AS,0.0,0.0,0.0,131.0,78.0,53.0
3,24_1408,Shea Crowther,CUR_FIE_IN,3/26/24,2024-04-02,2.0,1.0,1.0,0.0,0.0,0.0,SN,3.0,1.0,4.0,131.0,78.0,53.0
4,24_1314,Shea Crowther,CUR_FIE_IN,4/2/24,2024-04-09,0.0,1.0,0.0,0.0,0.0,0.0,LM,0.0,1.0,1.0,131.0,78.0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2090,24_2941,Marion Zuefle,ZUE_REE,10/29/24,2024-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3170.0,1801.0,1369.0
2091,24_2942,Marion Zuefle,ZUE_REE,11/5/24,2024-11-12,7.0,11.0,7.0,14.0,2.0,3.0,EW,14.0,25.0,39.0,3170.0,1801.0,1369.0
2092,24_2943,Marion Zuefle,ZUE_REE,11/5/24,2024-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3170.0,1801.0,1369.0
2093,24_3037,Marion Zuefle,ZUE_REE,11/12/24,2024-11-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3170.0,1801.0,1369.0
